<a href="https://colab.research.google.com/github/kaishijeng/MyGitHubtest/blob/master/Copy_of_edgecompiler_compat_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install tf-nightly==2.2.0.dev20200301

     |████████████████████████████████| 515.7MB 33kB/s 
     |████████████████████████████████| 460kB 45.6MB/s 
     |████████████████████████████████| 2.8MB 48.3MB/s 


In [0]:
import cv2
import numpy as np
import tensorflow as tf

In [5]:
tf.__version__

'2.2.0-dev20200301'

In [6]:
# following https://www.tensorflow.org/lite/performance/post_training_integer_quant
mnist_train, _ = tf.keras.datasets.mnist.load_data()
images = tf.cast(mnist_train[0], tf.float32)/255.0
mnist_ds = tf.data.Dataset.from_tensor_slices((images)).batch(1)

def representative_data_gen():
  for input_value in mnist_ds.take(50):
    payload = input_value[0].numpy()
    payload = cv2.resize(payload, (128, 128))
    payload = np.stack((payload, ) * 3, axis=-1)
    yield [np.expand_dims(payload, axis=0)]

11493376/11490434 [==============================] - 0s 0us/step


In [7]:
# MobileNetV2
def nn():
  base = tf.keras.applications.MobileNetV2(
            input_shape=(128, 128, 3),
            include_top=False,
            weights='imagenet'
  )
  x = tf.keras.layers.Conv2D(1, 3, activation='sigmoid')(base.output)
  return tf.keras.models.Model(inputs=base.input, outputs=x)

net = nn()
opti_b = tf.keras.optimizers.Adam()
net.compile(opti_b, loss='binary_crossentropy', metrics=['acc'])

9412608/9406464 [==============================] - 0s 0us/step


In [0]:
net.save('model.h5')

In [9]:
converter = tf.compat.v1.lite.TFLiteConverter.from_keras_model_file('model.h5')
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open('model.tflite', 'wb').write(tflite_model)

2864496

In [15]:
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!echo "deb https://packages.cloud.google.com/apt coral-edgetpu-stable main" | sudo tee /etc/apt/sources.list.d/coral-edgetpu.list
!sudo apt-get update 
!sudo apt-get -y install edgetpu-compiler

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   653  100   653    0     0  27208      0 --:--:-- --:--:-- --:--:-- 28391
OK
deb https://packages.cloud.google.com/apt coral-edgetpu-stable main
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://packages.cloud.google.com/apt coral-edgetpu-stable InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:9 http:

In [16]:
!edgetpu_compiler -s model.tflite

Edge TPU Compiler version 2.1.302470888

Model compiled successfully in 572 ms.

Input model: model.tflite
Input size: 2.73MiB
Output model: model_edgetpu.tflite
Output size: 2.75MiB
On-chip memory used for caching model parameters: 2.67MiB
On-chip memory remaining for caching model parameters: 5.05MiB
Off-chip memory used for streaming uncached model parameters: 0.00B
Number of Edge TPU subgraphs: 1
Total number of operations: 71
Operation log: model_edgetpu.log

Operator                       Count      Status

LOGISTIC                       1          Mapped to Edge TPU
ADD                            10         Mapped to Edge TPU
PAD                            5          Mapped to Edge TPU
QUANTIZE                       2          Mapped to Edge TPU
CONV_2D                        36         Mapped to Edge TPU
DEPTHWISE_CONV_2D              17         Mapped to Edge TPU
